In [86]:
import docx
import os
import xml.etree.ElementTree as ET

In [177]:
rtf_path = os.path.abspath('data/transcripts')
doc_path = os.path.abspath('data/transcripts/doc')
filename = "The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1)"

In [175]:
# import subprocess

# def convert_rtf_to_docx(rtf_file_path, docx_file_path):
#     subprocess.call(['unoconv', '-f', 'docx', '-o', docx_file_path, rtf_file_path])
#     print(os.path.abspath(docx_file_path))

# convert_rtf_to_docx("The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1).rtf", "The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1).docx")

In [178]:
import aspose.words as aw
doc = aw.Document("The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1).rtf")
doc.save("The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1).docx")

In [179]:
filename = "The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1).docx"
doc = docx.Document(filename)

In [216]:
filename.split(",")[0]

'The Bank of New York Mellon Corporation'

In [224]:
company = ""
title = ""
time = ""
currency = ""
note = ""
company_idx = -1
QA = ""
presentation = ""
for i, paragraph in enumerate(doc.paragraphs):
    print(paragraph.text, i)
    print(filename.split(",")[0], paragraph.text.lower())
    if i >=2 and paragraph.text.lower().strip().startswith(filename.split(",")[0]):
        company = paragraph.text
        company_idx = i
        
    elif company_idx !=-1 and i == company_idx +1:
        title = paragraph.text
    elif company_idx !=-1 and i == company_idx +2:
        time = paragraph.text
    elif company_idx !=-1 and i == company_idx +4:
        currency= paragraph.text
    elif company_idx !=-1 and i == company_idx +5:
        note= paragraph.text
    
    elif paragraph.text.strip().startswith("Question and Answer"):
        QA = paragraph.text
    elif paragraph.text.strip().startswith("Presentation "):
        presentation = paragraph.text
    

    

Evaluation Only. Created with Aspose.Words. Copyright 2003-2024 Aspose Pty Ltd. 0
The Bank of New York Mellon Corporation evaluation only. created with aspose.words. copyright 2003-2024 aspose pty ltd.
 1
The Bank of New York Mellon Corporation 
The Bank of New York Mellon Corporation NYSE:BK 2
The Bank of New York Mellon Corporation the bank of new york mellon corporation nyse:bk
FQ2 2023 Earnings Call Transcripts 3
The Bank of New York Mellon Corporation fq2 2023 earnings call transcripts
Tuesday, July 18, 2023 2:30 PM GMT 4
The Bank of New York Mellon Corporation tuesday, july 18, 2023 2:30 pm gmt
S&P Global Market Intelligence Estimates 5
The Bank of New York Mellon Corporation s&p global market intelligence estimates
Currency: USD 6
The Bank of New York Mellon Corporation currency: usd
Consensus as of  Jul-18-2023 3:14 AM GMT 7
The Bank of New York Mellon Corporation consensus as of  jul-18-2023 3:14 am gmt
 8
The Bank of New York Mellon Corporation 
 9
The Bank of New York Mellon

In [220]:
print(len(doc.paragraphs))
print(company, title, time, currency,  note, company_idx)

17
     -1


In [225]:
print(QA)

Question and Answer 
 
 
Operator 
 
[Operator Instructions] And our first question will come from Brennan Hawken with UBS. 
 
 
Brennan  Hawken 
UBS Investment Bank, Research Division 
 
Dermot, I'd like to start with which you just touched on, the unchanged NII outlook. Curious what you're seeing as far as deposit cost pressure. We've begun to see that emerge. There are peer firms of Bank of New York's that have flagged some upward pressure in the back book. How is it that you are able to avoid these pressures and keep on your NII outlook that you had provided earlier? 
 
 
Dermot William McDonogh 
Senior Executive VP & CFO 
 
Thanks for the question, Brennan. So I will accept the fact that the pressures are there, but I think the team is doing a great job managing both sides of the balance sheet in a very dynamic environment. So if we kind of look at the first half of the year, I think we're very pleased with where we've come out in the first half of the year and feel we're on a ver

In [181]:
tables = []
for table_index, table in enumerate(doc.tables):
        t = []
        for row_index, row in enumerate(table.rows):
            row_data = []

           
            for cell in row.cells:
                row_data.append(cell.text.strip())

            if all(element == "" for element in row_data):
                 continue
            t.append(row_data)
        if t== [['']] or t ==[]:
             continue
        tables.append(t)
        

In [210]:
def build_first_table(data):
    root = ET.Element("table",attrib={"id":"0", "name":"Earnings Estimates Comparison Table"})

    time_periods = ET.SubElement(root, "timePeriods")
    for period in data[0][1:]:
        ET.SubElement(time_periods, "period", name=period.replace("-", ""))

    metrics = ET.SubElement(root, "metrics")
    for metric_data in data[2:]:
        metric = ET.SubElement(metrics, "metric", name=metric_data[0])
        for i, value in enumerate(metric_data[1:]):
            period_name = data[0][i + 1]
            value_type = data[1][i + 1]
            ET.SubElement(metric, "value", period=period_name, type=value_type).text = value
    
    return root

def build_second_table(data):
    # clean duplicates   
    data = [list(dict.fromkeys(row)) for row in data]
    
    root = ET.Element("table",attrib={"id":"1","name":"EPS Normalized Comparison Table"})
    

    type_header = data[1]

    time_periods = ET.SubElement(root, "timePeriods") 
    metrics = ET.SubElement(root, "metrics") 
    metric = ET.SubElement(metrics, "metric", name="EPS Normalized")
    for row in data[2:]:
        period = row[0]
        ET.SubElement(time_periods, "period", name=period)
        for i, cell in enumerate(row[1:], 1):
            type_ = type_header[i]  
            if type_ in ["CONSENSUS", "ACTUAL", "SURPRISE"]:
                value_element = ET.SubElement(metric, "value")
                value_element.text = cell
                value_element.set("period", period)
                value_element.set("type", type_)

    return root

def build_third_table(data):
    id = 0
    root = ET.Element("CallParticipants")
         
    current_group = ''
    for row in data[1:]:
        
        row_data = ' '.join(row).strip()
        elements = row_data.split('\n \n \n')
        for element in elements:
            lines = element.split('\n')

            if len(lines) == 1 :
                current_group = lines[0].strip()
                
            if len(lines) > 1:

                name = lines[0].strip()
                position = lines[1].strip()

                person_element = ET.SubElement(root, "person", position=position, group=current_group, id = str(id))
                person_element.text = name
                id+=1
    return root

In [202]:
def prettify(element, indent='    ', level=0):
    
    if element:  
        if not element.text or not element.text.strip():
            element.text = '\n' + indent * (level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = '\n' + indent * level
    else:
        if level and (not element.tail or not element.tail.strip()):
            element.tail = '\n' + indent * level
    
    for subelement in element:
        prettify(subelement, indent, level + 1)

In [208]:
root = ET.Element("body")
sec1 = ET.Element("section", attrib={"name": "financial tables"})
t1 = build_first_table(tables[0])
t2 = build_second_table(tables[1])
t3 = build_third_table(tables[3])
sec1.append(t1)
sec1.append(t2)
t3.tag = "section"
t3.set("name", "call participants")
root.append(sec1)
root.append(t3)
prettify(root)


tree = ET.ElementTree(root)
tree.write("output.xml", encoding="utf-8", xml_declaration=True)

EXECUTIVES 
Dermot William McDonogh 
Senior Executive VP & CFO 
Marius  Merz 
Head of Investor Relations 
Robin Antony Vince 
President, CEO & Director 
ANALYSTS 
Alexander  Blostein 
Goldman Sachs Group, Inc., Research Division 
Betsy Lynn Graseck 
Morgan Stanley, Research Division 
Brennan  Hawken 
UBS Investment Bank, Research Division 
Brian Bertram Bedell 
Deutsche Bank AG, Research Division 
Ebrahim Huseini Poonawala 
BofA Securities, Research Division 
Gerard Sean Cassidy 
RBC Capital Markets, Research Division 
Glenn Paul Schorr 
Evercore ISI Institutional Equities, Research Division 
Kenneth Michael Usdin 
Jefferies LLC, Research Division Rajiv K. Bhatia 
Morningstar Inc., Research Division 
Robert Henry Wildhack 
Autonomous Research US LP 
Steven Joseph Chubak 
Wolfe Research, LLC
